In [7]:
import BioSimSpace as bss
import glob

In [9]:
import BioSimSpace as bss
import glob
ligand_path = "../inputs/ligands/"
ligand_files = glob.glob(ligand_path+"*.mol2")

ligands = []
ligand_names = []
for file in ligand_files:
    ligand_name = file.split("/")[-1].replace(".mol2", "")
    ligand_names.append(ligand_name)
    print(ligand_name)
    ligand = bss.IO.readMolecules(file)[0]
    ligands.append(ligand)
    
transformations, lomap_scores = bss.Align.generateNetwork(ligands, plot_network=True, names=ligand_names)

lig_h_8
lig_h_4
lig_h_13
lig_h_16
lig_h_9
lig_h_6
lig_h_15
lig_h_10
lig_h_7
lig_h_12
lig_h_3
lig_h_1
lig_h_5
lig_h_2
lig_h_14
lig_h_11
------------------------------


AlignmentError: Unable to compute MCS of molecules!